In [ ]:
import tensorflow as tf 
from tensorflow import keras    
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import read_signal as rs
import tqdm
from matplotlib.animation import FuncAnimation 

In [ ]:
EEG,labels=rs.read_file("../BraiNeoCare/Datasets/zenodo_eeg/eeg9.edf",9)

In [ ]:
model = keras.models.load_model("./GAT_model_97/cp_0192.ckpt/")
model.layers[-1].activation = None

In [ ]:
# mean=np.load("mean.npy")
# std=np.load("std.npy")

grad_model = keras.Model(model.inputs,[model.get_layer("gat_layer_2").output,model.output])

def PreprocesSignal(signal,mean,std):
    signal=(signal-mean)/std
    signal=np.expand_dims(signal,axis=-1)
    signal=np.expand_dims(signal,axis=0)
    return signal

def GradCAM(signal,model=grad_model):
    
    with tf.GradientTape() as tape:
        GAT_outputs, predictions = grad_model(signal)
        Class=predictions[:,0]

    grads = tape.gradient(Class, GAT_outputs)

    heatmap = GAT_outputs[0] *  tf.reduce_mean(grads, axis=(0,1,2))

    heatmap = tf.nn.relu(heatmap)/tf.reduce_max(heatmap)

    return heatmap.numpy(),tf.nn.sigmoid(predictions[0][0]).numpy()

In [ ]:
def plot_explainable_signal():

    l, u = 0, 384
    length=EEG.shape[1]
    hmap=np.empty((12,length))
    predicted_class=[]
    x1=PreprocesSignal(EEG)
    
    for r in tqdm.tqdm(range(0,length-384,384),desc="processed GRAD-CAM"):
        h,p=GradCAM(x1[:,:,l:u,:])
        resized_heatmap = cv.resize(h, (384,12), interpolation=cv.INTER_LINEAR)
        hmap[:,l:u]=resized_heatmap
        predicted_class.append(1 if p>=0.5 else 0)
        l+=384
        u+=384 

    return hmap,predicted_class

hmap,prediction=plot_explainable_signal()
prediction=np.array([prediction])
prediction=np.repeat(prediction,384,axis=1)

In [ ]:
channel_names=["Fp1-T3","T3-O1","Fp1-C3","C3-O1","Fp2-C4","C4-O2","Fp2-T4","T4-O2","T3-C3","C3-Cz","Cz-C4","C4-T4"]
fig,ax=plt.subplots(13,1,figsize=(20,30))
length=EEG.shape[1]
EEG1=(EEG-mean)/std
labels1=(labels-mean)/std
ax[0].plot(prediction[0],color='g',linewidth=2)
ax[0].set_title("Model Prediction - 1:Seizure, 0:Non-Seizure")
ax[0].set_ylim(-0.1,1.1)
ax[0].set_xlim(0,length)
for r in range(12):
    ax[r+1].plot(EEG1[r],color='k')
    ax[r+1].plot(labels1[r],color='r')
    im=ax[r+1].imshow(hmap[r].reshape(1,length),cmap='bwr',alpha=0.5,extent=[0,length,EEG1[r].min(),EEG1[r].max()],aspect='auto',vmax=1,vmin=0)
    ax[r+1].set_title(channel_names[r])

fig.tight_layout()
cbar=fig.colorbar(im,ax=ax,orientation='horizontal',pad=0.02,shrink=0.5)
cbar.set_label('Relevance')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

x=np.load('../BraiNeoCare/Datasets/GAT/Artifacts_removed/zenodo_data_consensus_4s_GAT.npy', mmap_mode='r')
y=np.load('../BraiNeoCare/Datasets/GAT/Artifacts_removed/zenodo_labels_consensus_4s_GAT.npy', mmap_mode='r')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
mean=x_train.mean()
std=x_train.std()    

In [ ]:
channel_names=["Fp1-T3","T3-O1","Fp1-C3","C3-O1","Fp2-C4","C4-O2","Fp2-T4","T4-O2","T3-C3","C3-Cz","Cz-C4","C4-T4"]
n=3
x=PreprocesSignal(x_test[n],mean,std)
print("True label:- ",y_test[n])
h,p=GradCAM(x)
print("prediction:- ",1 if p>=0.5 else 0)
resized_heatmap = cv.resize(h, (384,12), interpolation=cv.INTER_LINEAR)
fig,ax=plt.subplots(12,1,figsize=(20,30))
for r in range(12):
    ax[r].plot(x[0,r,:,0],color='k')
    im=ax[r].imshow(resized_heatmap[r].reshape(1,384),cmap='bwr',alpha=1,extent=[0,384,x[0,r,:,0].min(),x[0,r,:,0].max()],aspect='auto',vmax=1,vmin=0)
    ax[r].set_title(channel_names[r])   
fig.tight_layout()
cbar=fig.colorbar(im,ax=ax,orientation='horizontal',pad=0.02,shrink=0.5)
cbar.set_label('Relevance')
plt.show()